In [18]:
import sys, os
module_paths = [os.path.abspath('.'), os.path.abspath('..')]
for module_path in module_paths:
    if module_path not in sys.path:
        sys.path.append(module_path)

import numpy as np
import pandas as pd
import scanpy as sc
import networkx as nx

import torch
import torch_geometric as pyg

In [19]:
dataset = "L008" #marson, sciplex

# 1. initial
graph = "grn.pkl"
dimension = 8

In [ ]:
'''
# 2. updated
graph = "%s_grn.pkl" % dataset
dimension = 128
'''

In [20]:
adata = sc.read('../datasets/%s_prepped.h5ad' % dataset)
adata.var

,mito,n_cells,highly_variable,means,dispersions,dispersions_norm
PERM1,False,48,True,0.000282,1.064582,2.723460
AL645608.7,False,8,True,0.000048,1.515906,4.124676
TTLL10,False,136,True,0.000833,0.976748,2.320261
TNFRSF18,False,86547,True,0.938895,0.915269,5.125483
TNFRSF4,False,88190,True,1.004453,0.994387,5.840080
...,...,...,...,...,...,...
RPS4Y1,False,37645,True,0.490005,1.509542,20.917027
PRKY,False,16216,True,0.097039,0.925138,4.583006
USP9Y,False,21406,True,0.131560,0.896381,5.459471
DDX3Y,False,27168,True,0.205420,1.001347,9.897668


In [21]:
nodes = np.array(adata.var.index)
nodes

array(['PERM1', 'AL645608.7', 'TTLL10', ..., 'USP9Y', 'DDX3Y', 'EIF1AY'],
      dtype=object)

In [22]:
source_path = "./node/source/%s" % dataset
export_path = "./node/export/%s" % dataset
os.makedirs(source_path, exist_ok=True)
os.makedirs(export_path, exist_ok=True)

In [23]:
np.savetxt(os.path.join(source_path, "genes.txt"),
    np.expand_dims(nodes, 1), fmt="%s"
)

In [24]:
from gvci.utils.general_utils import Suppressor
from node.gene2vec import gene2vec

with Suppressor():
    gene2vec(os.path.abspath(source_path), os.path.abspath(export_path), 'txt', dimension=dimension)

In [25]:
nodes_emb = pd.read_csv(os.path.join(export_path, "gene2vec_dim_%s_iter_10.txt") % dimension, sep=" ", index_col=0, header=None)
nodes_emb

,1,2,3,4,5,6,7,8,9,10,...,119,120,121,122,123,124,125,126,127,128
0,,,,,,,,,,,,,,,,,,,,,
LINC00858,0.003542,-0.001215,0.001945,0.001278,-0.001829,0.001718,0.000143,-0.001030,-0.002358,0.002222,...,0.000305,0.001580,-0.002127,-0.002126,0.000488,0.002571,0.003811,-0.000152,-0.002099,0.000958
TRAV2,0.002181,-0.002596,0.001249,0.002189,0.001884,-0.001652,0.000192,0.002426,-0.000076,0.003001,...,0.000096,-0.000737,-0.000560,0.002024,-0.001201,-0.002491,-0.001386,-0.001832,-0.002688,0.002967
AC078883.2,-0.003033,-0.001809,0.001240,0.003574,-0.001829,0.001150,-0.003513,0.003745,-0.000522,-0.001208,...,0.002082,-0.002157,-0.001686,0.001767,0.002506,-0.003903,0.003278,0.000143,0.000004,0.001928
TRBV20-1,0.001685,0.003851,-0.003494,0.000167,0.003281,-0.002159,0.000452,0.002975,0.003568,-0.002158,...,0.000272,0.003697,0.003641,-0.000085,0.001998,-0.000632,0.001540,-0.000766,0.000373,-0.003807
AL356433.1,0.000107,0.002853,-0.001400,0.002992,0.001033,-0.001295,-0.001350,-0.000953,-0.001533,-0.003903,...,0.002857,-0.001940,0.000215,0.000149,-0.001111,-0.003449,0.000961,-0.002117,0.001884,-0.003056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACTBL2,0.002277,-0.003387,0.002414,-0.000591,-0.000002,0.002597,-0.002413,-0.003434,0.000297,-0.003735,...,-0.002573,-0.003697,-0.000489,0.002347,0.002833,-0.000664,0.001193,0.003360,-0.002272,-0.003755
AL365184.1,0.000186,-0.003339,0.002609,0.001022,0.001954,-0.003444,0.000040,-0.003451,0.000744,-0.003660,...,-0.000292,-0.002637,-0.002500,-0.001029,0.003164,-0.001360,-0.001486,0.003028,0.000205,0.000859
F7,-0.003686,-0.002746,-0.001082,0.000925,-0.001716,-0.003214,-0.000732,-0.000915,-0.002174,0.000795,...,-0.000083,0.003070,0.001057,-0.003901,-0.003571,0.001401,-0.002979,0.003123,-0.003853,-0.001550


In [26]:
nodes_emb_dict = nodes_emb.T.to_dict('list')

In [27]:
import pickle
'''
from src.utils.graph_utils import parse_grn

graph_df = pd.read_parquet('hg19_TFinfo_dataframe_gimmemotifsv5_fpr2_threshold_10_20210630.parquet')
nx_graph = parse_grn(graph_df, 'gene_short_name')

pickle.dump(nx_graph, open('./grn.pkl', 'wb'))
'''
with open(graph, 'rb') as f:
    nx_graph = pickle.load(f)

In [28]:
for n in nodes:
    if not nx_graph.has_node(n):
        nx_graph.add_node(n)
def filter_node(n):
    return n in nodes
nx_graph = nx.subgraph_view(nx_graph, filter_node)

In [29]:
nodes_dict = dict(zip(nodes, range(len(nodes))))

In [30]:
x = []
edge_index = []
for i, node in enumerate(nodes):
    x.append(nodes_emb_dict[node])

    edges = list(nx_graph.in_edges(node))
    edges = [(nodes_dict[n[0]], i) for n in edges]
    edge_index.extend(edges)

In [31]:
x = torch.Tensor(x)
x.size()

torch.Size([2048, 128])

In [32]:
edge_index = torch.LongTensor(edge_index)
edge_index.size()

torch.Size([25288, 2])

In [33]:
grn = pyg.data.Data(x, edge_index.t())
torch.save(grn, '%s_grn_%s.pth' % (dataset, dimension))

In [34]:
torch.load('%s_grn_%s.pth' % (dataset, dimension))

Data(x=[2048, 128], edge_index=[2, 25288])